In [1]:
import numpy as np
import pandas as pd
import warnings
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
import glob
import json
import tensorflow_hub as hub
import tensorflow_text
import tldextract   # Accurately separates a URL's subdomain, domain, and public suffix
from bs4 import BeautifulSoup
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report,confusion_matrix,accuracy_score
from sklearn.metrics import precision_score, recall_score
from sklearn.pipeline import Pipeline
from sklearn.metrics import roc_auc_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import RandomizedSearchCV
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.naive_bayes import MultinomialNB
from xgboost import XGBClassifier
from sklearn.linear_model import LogisticRegression
import re
from sklearn.metrics import roc_curve, roc_auc_score
from sklearn.feature_extraction.text import TfidfVectorizer,CountVectorizer
from imblearn.over_sampling import SMOTE,ADASYN
from collections import Counter
import nltk 
import spacy
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')

from wordcloud import WordCloud, ImageColorGenerator,STOPWORDS
import gensim
from gensim.parsing.preprocessing import STOPWORDS as gensim_stopwords
import copy
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords,wordnet
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer,LancasterStemmer
from sklearn.metrics import f1_score
from pprint import pprint
import warnings
warnings.filterwarnings('ignore')
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
pd.set_option('display.expand_frame_repr', False)
pd.set_option('display.max_colwidth', 1000)

2022-06-20 00:24:23.149705: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2022-06-20 00:24:23.351494: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /gpfs/software/ada/python/anaconda/2020.11/3.8/lib:/gpfs/software/ada/cuda/10.2.89/lib64:/gpfs/software/ada/cuda/10.2.89/lib/lib64
2022-06-20 00:24:23.351568: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
[nltk_data] Downloading package stopwords to
[nltk_data]     /gpfs/home/psc21zcu/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Download

#### Loading the ISOT Full Dataset  -- After combined fake and real news  in the previous notebook   "ISOT_ML_Classifiers_TfiDF_Tokeniser"

In [2]:
isot_full_df = pd.read_csv("Updated//ISOT_Combined_FullData.csv")

#### Dropping the columns 'title' , 'text' and 'date'

In [3]:
isot_full_df = isot_full_df.drop(columns = ['title','text', 'subject','date','title_length','body_length','date'])

In [4]:
isot_full_df=isot_full_df.reset_index(drop=True)

In [5]:
isot_full_df.head(2)

,class,fulltext
0,0,"Macron vows Caribbean rebuild as anger rises against European powers French President Emmanuel Macron vowed to quickly rebuild the islands of the French Caribbean during a visit on Tuesday meant to dispel anger at his government s response to Hurricane Irma, which killed at least 43 people in the region. The clutch of Caribbean islands hardest hit by the storm were mainly overseas territories belonging to Britain, France and the Netherlands, whose tens of thousands of residents are European Union citizens. The U.S. Virgin Islands were also hard hit. European countries and the United States have sent troops to deliver aid and provide security after the storm toppled homes and hospitals, but locals and tourists short of food or shelter say help was slow to arrive. Macron, who is also facing the first test at home of his resolve to reform the economy with a day of protests against his labor reforms, denied that authorities reacted too slowly. Basic services in the region were lost af..."
1,0,"Cruz, Rubio say Iran prisoner swap a 'dangerous precedent' Republican presidential candidates Ted Cruz and Marco Rubio praised Iran’s release of five detained Americans on Sunday, but sharply criticized the deal the White House made to win their freedom saying it would lead to more Americans being taken “hostage.” U.S. President Barack Obama pardoned three Iranian-Americans charged for violating sanctions against Iran, a lawyer for one of the men said, while prosecutors moved to drop charges against four Iranians outside the United States. Iran agreed to free five Americans including Rezaian and Saeed Abedini, an Iranian-American Christian pastor sentenced to eight years in prison in 2013 on charges of undermining Iran’s national security. Cruz, speaking to Fox News Sunday, said, “Praise God that the prisoners are coming home” but Iran got the better end of the prisoner swap. “We released seven terrorists who had helped Iran with their nuclear program, and we agreed not to..."


### BERT Embeddings

In [6]:
# https://www.tensorflow.org/text/tutorials/classify_text_with_bert
#https://androidkt.com/simple-text-classification-using-bert-in-tensorflow-keras-2-0/

In [7]:
texts=isot_full_df['fulltext']

In [21]:
# bert preprocessor - https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3
preprocessor = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3")
# bert encoder - https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-12_H-512_A-8/2
encoder = hub.KerasLayer("https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-12_H-512_A-8/2",trainable=True)

2022-06-20 00:16:47.412807: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /gpfs/software/ada/python/anaconda/2020.11/3.8/lib:/gpfs/software/ada/cuda/10.2.89/lib64:/gpfs/software/ada/cuda/10.2.89/lib/lib64
2022-06-20 00:16:47.413205: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublas.so.11'; dlerror: libcublas.so.11: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /gpfs/software/ada/python/anaconda/2020.11/3.8/lib:/gpfs/software/ada/cuda/10.2.89/lib64:/gpfs/software/ada/cuda/10.2.89/lib/lib64
2022-06-20 00:16:47.413505: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublasLt.so.11'; dlerror: libcublasLt.so.11: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH

#### First set

In [12]:
# preprocessing dataset  - First Set
inputs = preprocessor(texts[0:10000])
# feeding it to model for vectorization
outputs = encoder(inputs)

In [13]:
pooled_output = outputs["pooled_output"]      # [batch_size, 512].
sequence_output = outputs["sequence_output"]  # [batch_size, seq_length, 512].

In [14]:
# defining dataframe
bertf_df1=pd.DataFrame()

In [15]:
## Converting bert encoder sequence output to 1 dimension for ML Model training

for i in range(0,len(sequence_output)):
    b=sequence_output[i].numpy().sum(axis=0)
    bertf_df1=bertf_df1.append(pd.Series(b),ignore_index=True)
print('values added in dataframe')

values added in dataframe


In [16]:
bertf_df1.to_csv('Updated//bertf_df1.csv', index=False)

#### Second Set

In [9]:
# preprocessing dataset  - First Set
inputs2 = preprocessor(texts[10000:20000])
# feeding it to model for vectorization
outputs2 = encoder(inputs2)

In [10]:
pooled_output2 = outputs2["pooled_output"]      # [batch_size, 512].
sequence_output2 = outputs2["sequence_output"]  # [batch_size, seq_length, 512].

In [11]:
# defining dataframe
bertf_df2=pd.DataFrame()

In [12]:
## Converting bert encoder sequence output to 1 dimension for ML Model training

for i in range(0,len(sequence_output2)):
    b=sequence_output2[i].numpy().sum(axis=0)
    bertf_df2=bertf_df2.append(pd.Series(b),ignore_index=True)
print('values added in dataframe')

values added in dataframe


In [13]:
bertf_df2.to_csv('Updated//bertf_df2.csv', index=False)

#### Third Set

In [14]:
# preprocessing dataset  - First Set
inputs3 = preprocessor(texts[20000:30000])
# feeding it to model for vectorization
outputs3 = encoder(inputs3)

In [15]:
pooled_output3 = outputs3["pooled_output"]      # [batch_size, 512].
sequence_output3 = outputs3["sequence_output"]  # [batch_size, seq_length, 512].

In [16]:
# defining dataframe
bertf_df3=pd.DataFrame()

In [17]:
## Converting bert encoder sequence output to 1 dimension for ML Model training

for i in range(0,len(sequence_output3)):
    b=sequence_output3[i].numpy().sum(axis=0)
    bertf_df3=bertf_df3.append(pd.Series(b),ignore_index=True)
print('values added in dataframe')

values added in dataframe


In [18]:
bertf_df3.to_csv('Updated//bertf_df3.csv', index=False)

#### Fourth Set

In [9]:
# preprocessing dataset  - First Set
inputs4 = preprocessor(texts[30000:40000])
# feeding it to model for vectorization
outputs4 = encoder(inputs4)

In [10]:
pooled_output4 = outputs4["pooled_output"]      # [batch_size, 512].
sequence_output4 = outputs4["sequence_output"]  # [batch_size, seq_length, 512].

In [11]:
# defining dataframe
bertf_df4=pd.DataFrame()

In [12]:
## Converting bert encoder sequence output to 1 dimension for ML Model training

for i in range(0,len(sequence_output4)):
    b=sequence_output4[i].numpy().sum(axis=0)
    bertf_df4=bertf_df4.append(pd.Series(b),ignore_index=True)
print('values added in dataframe')

values added in dataframe


In [13]:
bertf_df4.to_csv('Updated//bertf_df4.csv', index=False)

### Combining all the above 4 sets into a single dataframe

In [8]:
bertf_df1 = pd.read_csv("Updated//bertf_df1.csv")
bertf_df2 = pd.read_csv("Updated//bertf_df2.csv")
bertf_df3 = pd.read_csv("Updated//bertf_df3.csv")
bertf_df4 = pd.read_csv("Updated//bertf_df4.csv")

In [9]:
# merging both props 
bertVectors_fulldf=pd.concat([bertf_df1,bertf_df2,bertf_df3,bertf_df4])
bertVectors_fulldf.reset_index(drop=True,inplace=True)

In [11]:
bertVectors_fulldf.shape

(38270, 512)

In [12]:
isot_full_df['class'].value_counts()

0    20820
1    17450
Name: class, dtype: int64

### Adding class labels

In [13]:
bertVectors_fulldf.insert(len(bertVectors_fulldf.columns),'class',isot_full_df['class'])

In [18]:
bertVectors_fulldf.head(2)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,...,263,264,265,266,267,268,269,270,271,272,273,274,275,276,277,278,279,280,281,282,283,284,285,286,287,288,289,290,291,292,293,294,295,296,297,298,299,300,301,302,303,304,305,306,307,308,309,310,311,312,313,314,315,316,317,318,319,320,321,322,323,324,325,326,327,328,329,330,331,332,333,334,335,336,337,338,339,340,341,342,343,344,345,346,347,348,349,350,351,352,353,354,355,356,357,358,359,360,361,362,363,364,365,366,367,368,369,370,371,372,373,374,375,376,377,378,379,380,381,382,383,384,385,386,387,388,389,390,391,392,393,394,395,396,397,398,399,400,401,402,403,404,405,406,407,408,409,410,411,412,413,414,415,416,417,418,419,420,421,422,423,424,425,426,427,428,429,430,431,432,433,434,435,436,437,438,439,440,441,442,443,444,445,446,447,448,449,450,451,452,453,454,455,456,457,458,459,460,461,462,463,464,465,466,467,468,469,470,471,472,473,474,475,476,477,478,479,480,481,482,483,484,485,486,487,488,489,490,491,492,493,494,495,496,497,498,499,500,501,502,503,504,505,506,507,508,509,510,511,class
0,34.076153,-29.068226,57.540318,-51.954987,3.320737,-5.768854,79.962100,39.567856,-17.904541,-13.444562,7.104392,-55.4440,-24.303596,-44.187996,2.927974,53.233093,50.92992,-43.447510,-89.645250,22.505268,-60.829258,-20.836334,-0.965031,31.152092,-11.122220,-25.034784,4.409427,26.100441,-25.010174,-74.720795,53.209137,-81.84636,50.065517,-45.970085,30.723158,27.164768,-28.826418,-6.222152,6.740042,40.17959,44.086067,19.706493,20.024275,-64.465230,59.972400,-8.002469,13.231766,92.479520,59.146214,36.834324,-39.294914,38.488094,-15.634628,5.950771,64.343360,-57.710840,-31.729551,17.203966,6.631249,42.381054,2.142177,8.529372,-37.259970,6.108812,-27.347390,26.856197,-21.978996,-52.820110,53.621487,2.217431,81.62559,19.125967,-12.114867,114.19409,22.398640,-26.302088,9.557514,-63.690918,-6.997769,-72.960580,-62.32190,38.198235,-22.560627,58.402110,6.116537,-55.134212,10.474427,-24.749079,24.216830,76.149510,-57.48442,44.822865,10.780519,-26.609621,-5.474360,-58.219970,-2.624526,-48.936640,-8.839552,115.739075,11.119407,43.98946,64.425964,-41.28047,26.402365,47.93493,-28.618982,-32.531040,25.251757,7.147524,-20.038187,-6.329055,-82.623290,-19.123924,19.217548,24.678738,-46.619442,-16.176466,53.027214,-56.030000,-5.062048,-20.987423,2.937074,-38.340145,10.470112,59.40322,-51.526802,22.722168,9.580302,71.415770,-54.483715,-29.626957,-5.157812,-62.49628,38.692795,-0.546269,-5.400824,-69.03352,34.485413,34.81170,4.540720,14.029618,1.117358,67.148766,54.817284,-21.200008,-47.789494,38.494070,-79.732940,37.797665,-44.171960,42.925446,31.51484,-22.553644,33.205990,-22.327332,24.549444,13.433725,-25.152117,22.379246,8.82339,-5.749085,-32.831110,14.075165,-13.587171,-24.091106,3.647905,-49.28312,-73.42274,34.545450,121.210620,6.579218,-14.331875,25.185430,-0.427416,25.58867,32.091415,60.210037,114.56279,-18.183249,109.43097,4.606071,-816.7640,34.536625,-1.968195,-28.142189,-35.985890,-26.224033,-14.747208,-44.331577,40.095245,72.64052,22.202890,44.85098,45.743100,-17.891928,-69.220380,9.521957,-46.55743,38.384500,-22.983368,36.421482,-1.850905,53.13959,27.446419,54.202442,-41.3668

In [14]:
bertVectors_fulldf['class'].value_counts()

0    20820
1    17450
Name: class, dtype: int64

In [ ]:
bertVectors_fulldf.to_csv('Updated//bertVectors_fulldf_ISOT.csv', index=False)

### Loading dataframe BertVectors

In [16]:
bertVectors_fulldf = pd.read_csv("Updated//bertVectors_fulldf_ISOT.csv")

### Splitting dataset into train and test

In [17]:
X=bertVectors_fulldf.drop('class',axis=1)
y=bertVectors_fulldf['class']

#### Using StratifiedKFold

In [18]:
from sklearn.model_selection import StratifiedKFold
kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=1)

In [21]:
for i, j in kfold.split(X, y):
    # select rows
    X_train, X_test = X.iloc[i], X.iloc[j]
    y_train, y_test = y.iloc[i], y.iloc[j]

In [22]:
X_test.shape

(7654, 512)

#### Helper Functions

In [23]:
import timeit
from timeit import default_timer as timer
from datetime import timedelta
 


def metrics(X_test,y_test,clf):
    predictions=clf.predict(X_test)
    #predictions=(clf.predict_proba(X_test)[:,1] >= 0.3).astype(bool)
    print("confusion_matrix :")
    print(confusion_matrix(y_test,predictions))
    print("Accuracy Score :")
    print(accuracy_score(y_test, predictions))
    print("Classification Report :")
    print(classification_report(y_test, predictions))
    print("F1 score :")
    print(f1_score(y_test, predictions))
    print("ROC AUC Score")
    y_pred_proba = clf.predict_proba(X_test)
    print(roc_auc_score(y_test, y_pred_proba[:,1]) )
    print("------------------------------")

    
def model_comparison_table(X_test,y_test,classifier_list):
    dict_clf={}
    for clf_name,clf in classifier_list:
        predictions=clf.predict(X_test)
        y_pred_proba = clf.predict_proba(X_test)
        accuracy=accuracy_score(y_test, predictions)
        precision=precision_score(y_test,predictions,average='macro').round(2)
        recall=recall_score(y_test,predictions)
        f1score=f1_score(y_test,predictions).round(2)
        ROC_AUC=roc_auc_score(y_test, y_pred_proba[:,1])
        dict_clf[clf_name]=[accuracy,precision,recall,f1score,ROC_AUC]
    df_models_scores = pd.DataFrame(dict_clf, index=['Accuracy', 'Precision', 'Recall', 'F1 Score','roc_auc_score'])
    return df_models_scores

In [24]:
#### Helper Functions

import timeit
from timeit import default_timer as timer
from datetime import timedelta
 


def metrics(X_test,y_test,clf):
    predictions=clf.predict(X_test)
    #predictions=(clf.predict_proba(X_test)[:,1] >= 0.3).astype(bool)
    print("confusion_matrix :")
    print(confusion_matrix(y_test,predictions))
    print("Accuracy Score :")
    print(accuracy_score(y_test, predictions))
    print("Classification Report :")
    print(classification_report(y_test, predictions))
    print("F1 score :")
    print(f1_score(y_test, predictions))
    print("ROC AUC Score")
    y_pred_proba = clf.predict_proba(X_test)
    print(roc_auc_score(y_test, y_pred_proba[:,1]) )
    print("------------------------------")

    
def model_comparison_table(X_test,y_test,classifier_list):
    dict_clf={}
    for clf_name,clf in classifier_list:
        predictions=clf.predict(X_test)
        y_pred_proba = clf.predict_proba(X_test)
        accuracy=accuracy_score(y_test, predictions)
        precision=precision_score(y_test,predictions,average='macro').round(2)
        recall=recall_score(y_test,predictions)
        f1score=f1_score(y_test,predictions).round(2)
        ROC_AUC=roc_auc_score(y_test, y_pred_proba[:,1])
        dict_clf[clf_name]=[accuracy,precision,recall,f1score,ROC_AUC]
    df_models_scores = pd.DataFrame(dict_clf, index=['Accuracy', 'Precision', 'Recall', 'F1 Score','roc_auc_score'])
    return df_models_scores

## Random Forest

In [25]:
# Instantiate and fit a basic Random Forest model on top of the vectors
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(random_state=0)
rf_model = rf.fit(X_train, y_train)

In [26]:
# Use the trained model to make predictions on the test data
y_pred = rf_model.predict(X_test)
print("Accuracy score : {}".format(accuracy_score(y_test, y_pred)))
print("Confusion matrix : \n {}".format(confusion_matrix(y_test, y_pred)))
print("Classification Report")
print(classification_report(y_test, y_pred))

precision = precision_score(y_test, y_pred)
print("Precision : {}".format(precision))
recall = recall_score(y_test, y_pred)
print("Recall : {}".format(recall))

Accuracy score : 0.9380715965508231
Confusion matrix : 
 [[3992  172]
 [ 302 3188]]
Classification Report
              precision    recall  f1-score   support

           0       0.93      0.96      0.94      4164
           1       0.95      0.91      0.93      3490

    accuracy                           0.94      7654
   macro avg       0.94      0.94      0.94      7654
weighted avg       0.94      0.94      0.94      7654

Precision : 0.9488095238095238
Recall : 0.9134670487106017


### Support Vector Machine

In [39]:
svm_clf=SVC(random_state=0,probability=True)

In [40]:
svm_clf.fit(X_train, y_train)

pred = svm_clf.predict(X_test)
print("Accuracy score : {}".format(accuracy_score(y_test, pred)))
print("Confusion matrix : \n {}".format(confusion_matrix(y_test, pred)))
print("Classification Report")
print(classification_report(y_test, pred))

Accuracy score : 0.9760909328455709
Confusion matrix : 
 [[4089   75]
 [ 108 3382]]
Classification Report
              precision    recall  f1-score   support

           0       0.97      0.98      0.98      4164
           1       0.98      0.97      0.97      3490

    accuracy                           0.98      7654
   macro avg       0.98      0.98      0.98      7654
weighted avg       0.98      0.98      0.98      7654



## KNN

In [29]:
#Create KNN Object.
knn_clf = KNeighborsClassifier()

In [30]:
knn_clf.fit(X_train,y_train)

pred = knn_clf.predict(X_test)
print("Accuracy score : {}".format(accuracy_score(y_test, pred)))
print("Confusion matrix : \n {}".format(confusion_matrix(y_test, pred)))
print("Classification Report")
print(classification_report(y_test, pred))

Accuracy score : 0.9517899137705774
Confusion matrix : 
 [[4064  100]
 [ 269 3221]]
Classification Report
              precision    recall  f1-score   support

           0       0.94      0.98      0.96      4164
           1       0.97      0.92      0.95      3490

    accuracy                           0.95      7654
   macro avg       0.95      0.95      0.95      7654
weighted avg       0.95      0.95      0.95      7654



### Xgboost

In [31]:
#Create KNN Object.
xg_clf=XGBClassifier(random_state=0)

xg_clf.fit(X_train,y_train)

pred = xg_clf.predict(X_test)
print("Accuracy score : {}".format(accuracy_score(y_test, pred)))
print("Confusion matrix : \n {}".format(confusion_matrix(y_test, pred)))
print("Classification Report")
print(classification_report(y_test, pred))


Accuracy score : 0.9668147373922132
Confusion matrix : 
 [[4065   99]
 [ 155 3335]]
Classification Report
              precision    recall  f1-score   support

           0       0.96      0.98      0.97      4164
           1       0.97      0.96      0.96      3490

    accuracy                           0.97      7654
   macro avg       0.97      0.97      0.97      7654
weighted avg       0.97      0.97      0.97      7654



### Naive Bayes

In [32]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.naive_bayes import GaussianNB
nb_clf=GaussianNB()
#p = Pipeline([('Normalizing',MinMaxScaler()),('MultinomialNB',MultinomialNB())])
nb_clf.fit(X_train,y_train) 

pred = nb_clf.predict(X_test)
print("Accuracy score : {}".format(accuracy_score(y_test, pred)))
print("Confusion matrix : \n {}".format(confusion_matrix(y_test, pred)))
print("Classification Report")
print(classification_report(y_test, pred))

Accuracy score : 0.9178207473216619
Confusion matrix : 
 [[3895  269]
 [ 360 3130]]
Classification Report
              precision    recall  f1-score   support

           0       0.92      0.94      0.93      4164
           1       0.92      0.90      0.91      3490

    accuracy                           0.92      7654
   macro avg       0.92      0.92      0.92      7654
weighted avg       0.92      0.92      0.92      7654



### Logistic Regression

In [33]:
from sklearn.linear_model import LogisticRegression

In [34]:
logticreg_clf=LogisticRegression()

logticreg_clf.fit(X_train,y_train)

pred = logticreg_clf.predict(X_test)
print("Accuracy score : {}".format(accuracy_score(y_test, pred)))
print("Confusion matrix : \n {}".format(confusion_matrix(y_test, pred)))
print("Classification Report")
print(classification_report(y_test, pred))

Accuracy score : 0.9822315129344134
Confusion matrix : 
 [[4097   67]
 [  69 3421]]
Classification Report
              precision    recall  f1-score   support

           0       0.98      0.98      0.98      4164
           1       0.98      0.98      0.98      3490

    accuracy                           0.98      7654
   macro avg       0.98      0.98      0.98      7654
weighted avg       0.98      0.98      0.98      7654



### Consolidating all classifiers accuracy scores

In [41]:
classifiers = [('Logistic Regression',logticreg_clf),('Naive Bayes',nb_clf),('Xgboost',xg_clf),
              ('KNN',knn_clf),('Random Forest',rf_model),("SVM",svm_clf)]

In [42]:
df_model_scores=model_comparison_table(X_test,y_test,classifiers)
df_model_scores.head(20).T.sort_values(by='Accuracy', ascending=False)

,Accuracy,Precision,Recall,F1 Score,roc_auc_score
Logistic Regression,0.982232,0.98,0.980229,0.98,0.998288
SVM,0.976091,0.98,0.969054,0.97,0.997531
Xgboost,0.966815,0.97,0.955587,0.96,0.995014
KNN,0.951790,0.95,0.922923,0.95,0.984113
Random Forest,0.938072,0.94,0.913467,0.93,0.984804
Naive Bayes,0.917821,0.92,0.896848,0.91,0.968778
